# Decision Trees

Able to perform regression and classification task and also give multiple outputs. These are fundamental component of Random Forests, one of the most powerful machine learning tools in use.

## Training and Visualizing a Decision Tree

We'll start with a decision tree classifier.

In [2]:
import sklearn
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] #Petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

We can visualize our tree using export_graphviz()

In [6]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file="iris_tree.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

Use the command "dot -Tpng iris_tree.dot -o iris_tree.png" to change the .dot to a .png

## Making Predictions

Root nodes at depth 0 indicate the entry point of the decision tree. A leaf node has no children and will return a class or the probability of belonging to multiple classes.

One of decision trees many qualities is ease of visualization and they require little preperation. They do not require feature scaling or centering.

A node's value attribute will give the number of training instances of each class. The gini attribute is the "purity" of the node (a "pure" node vill have a score of 0). The score is computed like so: 

$$G_i = 1 - \sum_{k=1}^{n} p_{i, k}^2$$ 

$p_{i, k}$ is the ratio of class _k_ to all the instances in the _i_<sup>th</sup> node.

The algorithm we use to create our treee (CART algorithm) only produces binary trees. Other algorithms like ID3 can create trees with more than two children.

Models that are easy to interpret and understand are known as _white box models_. Decision trees are a member of such models. These contrast with _black box models_ like neural networks which make predictions that are much more difficult to interpret.

## Estimating Class Probabilities

Decision trees make predictions much like previously examined classifiers. It traverses the tree with the instance data and once it reaches a leaf, it ourputs the probability of belonging to a certain class or classes.

In [8]:
tree_clf.predict_proba([[5, 1.5]])

array([[ 0.        ,  0.90740741,  0.09259259]])

In [9]:
tree_clf.predict([[5, 1.5]])

array([1])

The probability values would not change if we were to increase the petal width beyond 6cm even though it would appear to be iris-virginica instead of the predicted iris-versicolor.

## The CART Training Algorithm

Scikit-learn uses the _Classifications and Regression Tree_ algorithm to train Decision Trees. The algorithm works by splitting the training data into two subsets based on a single feature _k_ and threshold _t<sub>k</sub>_. The algorithm finds the feature-threshold pair that produces the purest subsets (weighted by size). The CART cost function is defined like so:

$$J(k, t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right}$$

Here, $G_{left/right}$ measures the purity of the left/right subset and $m_{left/right}$ measures the number of instances in each subset. The algorithm will stop recursing once the depth _max depth_ has been reached or there are no splits which will reduce subset impurity. Other control hyperparameters exist like min_samples_split, min_samples_leaf, min_weight_fraction_leaf, and max_leaf_nodes. 

Finding the optimal tree is an $O(\exp(m))$ problem so we use a greedy solution to improve runtime.

## Computational Complexity

Making predictions is done in $O(\log_2(m))$. Training time will usually compare against all features (unless the max_features hyperparameter is used) resulting in a training time of $O(n \times m \log_2(m))$.

## Gini Impurity or Entropy?

In machin learning, entropy is used as an impurity measure: a set's entropy is zero when the set contains just one class. This is expressed as:

$$H_i = - \sum_{k=1, p_{i, k \neq 0}}^{n} p_{i,k} \log(p_{i,k})$$

Choosing entropy over Gini impurity usually doesn't result in a large change. When the difference is large, Gini impurity will isolate frequent classes while entropy will create more balanced trees.

## Regularization Hyperparameters

Decision trees make no assumptions about the data and are extrememly prone to overfitting to the training data. A model without specified parameters is called a _nonparametric model_. A _paramatric model_ has a limited degree of freedom and carries a reduced risk of overfitting and an increased risk of underfitting.

We can reduce the risk of overfitting by using restricting the Decision Tree's freedom (regularizing). The regularization paramters depend on the specific algorithm used but one can almost always restrict the maximum depth.

The scikit-learn DecisionTreeClassifier carries other parameters to regularize the tree: min<span>\_</span>samples<span>\_</span>split (specifies the number of samples required to split a node), min<span>\_</span>samples<span>\_</span>leaf (specifies number of samples a leaf node must have), min<span>\_</span>weight<span>\_</span>fraction<span>\_</span>leaf (same as min<span>\_</span>samples<span>\_</span>leaf but expressed as a fraction of total number of weighted instances), and max<span>\_</span>features (maximum number of features to be evaluated. Increasing min<span>\_\*</span> and decreasing max<span>\_\*</span> will regularize the model.

## Regression

We can build a Decision Tree regressor like so:

In [10]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

Instead of predicting a class, the tree will now predict a value equal to the average target value of every instance in the node. The CART algorithm will now attempt to minimise the MSE of the instances in each node. The CART cost function now looks like this:

$$J(k, t_k) = \frac{m_{left}}{m}MSE_{left} + \frac{m_{right}}{m}MSE_{right}$$

Where $MSE_{node} = \sum_{i \in node}(\hat{y}_{node} - y^{(i)})^2$ and $\hat{y}_{node} = \frac{1}{m_{node}} \sum_{i \in node}y^{(i)}$

## Instability

Decision trees have some drawbacks. They usually create orthagonal decision boundaries and are vulnerable to rotated data. One can limit this with PCA (Chapter 8) which will better orient the data.

More generally, Decision trees are extremely sensitive to small variations in the training data. It's even possible to get a completely different tree from the same training data.